# Imports 

In [1]:
import pandas as pd
import numpy as np

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` 

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [2]:
sd_df = pd.read_csv("../../../public_data/sd_df.csv")
naics_df = pd.read_csv("../../../public_data/naics_df.csv")

In [3]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_sector` column in the San Diego business data as the join key
- Use the `naics_twodig` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [4]:
sd_cen_inner = pd.merge(sd_df,
                       naics_df,
                       how = "inner",
                       left_on = "naics_code",
                       right_on = "naics")


In [5]:
"""In our original data, there were {n_orig} rows; \
now after dropping ones without a naics code \
in our current census data there are {n_new} rows\
""".format(n_orig = sd_df.shape[0], 
          n_new = sd_cen_inner.shape[0])

## which naics codes got lost
## subset to account keys not in the 
## inner join and to the naics columns
lost_merge = sd_df.loc[~sd_df.account_key.isin(sd_cen_inner.account_key),
                     ['naics_code', 'naics_description', 'naics_nchar']].drop_duplicates()

lost_merge.head()
lost_merge.naics_nchar.value_counts()


'In our original data, there were 353 rows; now after dropping ones without a naics code in our current census data there are 52 rows'

,naics_code,naics_description,naics_nchar
1,5411,LEGAL SERVICES,4
4,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4
12,7221,FULL-SERVICE RESTAURANTS,4
13,48841,MOTOR VEHICLE TOWING,5
14,54111,OFFICES OF LAWYERS,5


5    56
4    26
3    16
6     9
2     7
Name: naics_nchar, dtype: int64

## "Left join"- retain all sd businesses even if naics code isn't in the naics_details df

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge]
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix


In [6]:
## try merging 
sd_cen_left = pd.merge(sd_df,
                       naics_df,
                       how = "left",
                       left_on = "naics_code",
                       right_on = "naics",
                    indicator = "naics_merge_status",
                    suffixes = ["_sd", "_census"])
sd_cen_left.naics_merge_status.value_counts(normalize = True)

## look at sample of ones that didnt merge
sd_cen_left[sd_cen_left.naics_merge_status == "left_only"].sample(5)


left_only     0.859459
both          0.140541
right_only    0.000000
Name: naics_merge_status, dtype: float64

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
5,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
243,2018021800,SPRINGHILL SUITES SAN DIEGO RANCHO BERNARDO,cd_5,45,RETAIL TRADE,2,NaN,NaN,left_only
299,2020006672,TEXLA TECHNOLOGY,cd_7,54133,ENGINEERING SERVICES,5,NaN,NaN,left_only
352,2021008299,RED DOOR ESCAPE ROOM LLC,cd_7,7139,OTHER AMUSEMENT & RECREATION INDUSTRIES,4,NaN,NaN,left_only
141,2008010814,SANTE FE SHOPPING DEPOT LTD,cd_3,5313,ACTIVITIES RELATED TO REAL ESTATE,4,NaN,NaN,left_only


## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator---`is_lost` if the merge indicator is equal to "left_only"
- Group by `council_district` and use the shortcut of taking the mean of a True/False indicator to find the proportions in order to find the proportion lost in the merge (so in the left join, ones that failed to match to `naics_df`) by council_district


In [7]:
sd_cen_left['is_lost'] = sd_cen_left.naics_merge_status == "left_only"
sd_cen_left.is_lost.value_counts()

sd_cen_left.groupby('council_district')['is_lost'].mean()

## can see this is likely because low N
sd_cen_left.groupby('council_district').agg({'is_lost': np.mean,
                                             'account_key': 'nunique'}).reset_index()

True     318
False     52
Name: is_lost, dtype: int64

council_district
cd_1    0.880597
cd_2    0.804348
cd_3    0.812500
cd_4    1.000000
cd_5    0.916667
cd_6    0.859649
cd_7    0.918919
cd_8    0.933333
cd_9    1.000000
Name: is_lost, dtype: float64

,council_district,is_lost,account_key
0,cd_1,0.880597,67
1,cd_2,0.804348,42
2,cd_3,0.812500,103
3,cd_4,1.000000,4
4,cd_5,0.916667,24
5,cd_6,0.859649,54
6,cd_7,0.918919,36
7,cd_8,0.933333,15
8,cd_9,1.000000,8


## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were oftentimes not 6-digits long, while the Census ones were always 6 digits

You wonder if this is an issue where 0's in some of the SD's data naics codes got cutoff (eg 540000 became 54) and if adding these lagging zeros would improve the merge rate in a left join

- Using one of two approaches, pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits: (1) str.pad in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html); (2) for more of a challenge, write your own function that checks the # of digits in the naics code string and pads with 0's at the end up to 6 digits and use row-wise apply---`df.apply(funcname, axis = 1)`---to execute it
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [8]:
## auto way
sd_df['naics_padded_auto'] = sd_df.naics_code.str.pad(width = 6,
                                                     side = "right",
                                                     fillchar = "0")


In [9]:
## function way- could also parametrize
## the colnames as second and third args
def pad_naics(one_row):
    
    if one_row.naics_nchar == 2:
        padded_n = one_row.naics_code + "0000"
    elif one_row.naics_nchar == 3:
        padded_n = one_row.naics_code + "000"
    elif one_row.naics_nchar == 4:
        padded_n = one_row.naics_code + "00"
    elif one_row.naics_nchar == 5:
        padded_n = one_row.naics_code + "0"
    else:
        padded_n = one_row.naics_code
    return(padded_n)

In [10]:
sd_df['naics_padded_func'] = sd_df.apply(pad_naics, axis = 1)

## re-merge and check match rate
lj_update = pd.merge(sd_df,
                       naics_df,
                       how = "left",
                       left_on = "naics_padded_auto",
                       right_on = "naics",
                    indicator = "naics_merge_status_padded",
                    suffixes = ["_sd", "_census"])
lj_update.naics_merge_status_padded.value_counts(normalize = True)

## create indicator for match with padded
## not match with non-padded
lost_r1_accountkey = sd_cen_left[sd_cen_left.is_lost].account_key

both          0.53139
left_only     0.46861
right_only    0.00000
Name: naics_merge_status_padded, dtype: float64

In [11]:
lj_update['is_new_match'] = np.where((lj_update.account_key.isin(lost_r1_accountkey)) & 
                            (lj_update.naics_merge_status_padded == "both"), True, False)
lj_update.is_new_match.value_counts()


False    261
True     185
Name: is_new_match, dtype: int64

In [12]:
## compare descriptions
lj_update.loc[lj_update.is_new_match,
              ['dba_name',
               'naics_description_sd', 'naics_description_census']].sample(n = 5,
                                                                          random_state = 
                                                                          52092)

## seems correct

,dba_name,naics_description_sd,naics_description_census
204,INTELLIGENT BLENDS,COFFEE & TEA MFG,Coffee and Tea Manufacturing
345,AR WORKSHOP SAN DIEGO,"GIFT, NOVELTY & SOUVENIR STORES","Gift, Novelty, and Souvenir Stores"
285,THRIVE AFFORDABLE VET CARE,VETERINARY SERVICES,Veterinary Services
14,LATHAM & WATKINS LLP,OFFICES OF LAWYERS,Offices of Lawyers
326,GS MISSION GORGE PROJECT OWNER LP,LESSORS OF RESIDENTIAL BUILDINGS & DWELLINGS,Lessors of Residential Buildings and Dwellings
